In [2]:
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] # 设置字体
plt.rcParams["axes.unicode_minus"]=False # 正常显示负号
from database import * 

from pyecharts.charts import Bar, Line, Grid
import pyecharts.options as opts


In [3]:
sql = 'select score, count(*) count, avg(comment) avg_comment from movie group by score'
score, count, avg = execute_sql(sql, 3)
score = [str(i) for i in score]
avg = [round(i) for i in avg]
# plt.subplot(1, 2, 1)
# plt.bar(score, count, width=0.03)
# plt.title('评分-数量')

# plt.subplot(1, 2, 2)
# plt.plot(score, avg)
# plt.title('评分-平均评论数')

bar = (
        Bar()
        .add_xaxis(score)
        .add_yaxis('数量', count)
        .set_global_opts(
                title_opts=opts.TitleOpts(title='各评分的数量')
         )
)
line = (
        Line()
        .add_xaxis(score)
        .add_yaxis('平均评论数', avg)
        .set_global_opts(
                title_opts=opts.TitleOpts(title='各评分对应的平均评论数', pos_top='52%'),
                legend_opts=opts.LegendOpts(pos_top='52%')
        )
)

grid = (
        Grid(init_opts=opts.InitOpts(height="600px", width='1000px'))
        .add(bar, opts.GridOpts(pos_bottom='55%'))
        .add(line, opts.GridOpts(pos_top='60%'))
)
grid.render_notebook()

In [4]:
bar = (
        Bar(init_opts=opts.InitOpts(height="500px", width='1000px'))
        .add_xaxis(score)
        .add_yaxis('数量', count, yaxis_index=0, z=0,
                label_opts=opts.LabelOpts(is_show=True),
                color='MediumTurquoise')
        .extend_axis(yaxis=opts.AxisOpts(min_= 0))
        .set_global_opts(
                title_opts=opts.TitleOpts(title='评分 - 数量/平均评论数')
         )
)
line = (
        Line()
        .add_xaxis(score)
        .add_yaxis(
                '平均评论数', 
                avg, 
                yaxis_index=1, 
                symbol="circle",
                linestyle_opts=opts.LineStyleOpts(color="LimeGreen", width=4, type_="dashed"),
                itemstyle_opts=opts.ItemStyleOpts(
                        border_width=5, border_color="MediumAquamarine", color="white"
                ),
        )
        .set_global_opts(
                
        )
)
bar.overlap(line)
bar.render_notebook()
# line.render_notebook()

In [5]:
sql =  'select p.name, count(*) count, truncate(avg(score), 2) avg_score\
        from place p left outer join movie_place mp on p.id = mp.place_id join movie m on m.id = mp.movie_id\
        group by p.name\
        order by count desc'
name, count, avg_score = execute_sql(sql, 3)

bar = (
        Bar(init_opts=opts.InitOpts(height="500px", width='1000px'))
        .add_xaxis(name[:10])
        .add_yaxis('数量', count[:10], xaxis_index=0,)
        .reversal_axis()
        .extend_axis(xaxis=opts.AxisOpts(min_= min(avg_score[:10]) - 0.1))
        .set_global_opts(title_opts=opts.TitleOpts(title='各国家的入选数量', subtitle='取数量前十名'))
)

bar_add = (
        Bar()
        .add_xaxis(name)
        .add_yaxis('平均评分', avg_score[:10], xaxis_index=1)
)
bar.overlap(bar_add)
bar.render_notebook()

In [9]:
sql =  'select t.name name, count(*) count, truncate(avg(score), 2) avg_score\
        from type t left outer join movie_type mt on t.id = mt.type_id join movie m on m.id = mt.movie_id\
        group by t.name \
        order by count desc'
        
name, count, avg_score = execute_sql(sql, 3)
bar = (
        Bar(init_opts=opts.InitOpts(height="500px", width='1000px'))
        .add_xaxis(name)
        .add_yaxis('数量', count, category_gap="15%", yaxis_index=0,)
        # .add_yaxis('平均评分', avg_score)
        .extend_axis(yaxis=opts.AxisOpts(min_= min(avg_score) - 0.1))
        # .reversal_axis()
        .set_global_opts(
                xaxis_opts=opts.AxisOpts(
                        axislabel_opts=opts.LabelOpts(font_size=9, rotate=-30)
                ),
                
                title_opts=opts.TitleOpts(title='各类型的电影数量, 均分', subtitle=''),
                
                
        )
        
)
bar_add = (
        Bar()
        .add_xaxis(name)
        .add_yaxis('均分', avg_score, yaxis_index=1)
)
bar.overlap(bar_add)
bar.render_notebook()